
<br>
<font>
<div dir=ltr align=center>
<font color=0F5298 size=10>
    Deep Learning - HW4 <br>
<font color=2565AE size=5>
    Electrical Engineering Department <br>
    winter 2024<br>
<font color=3C99D size=5>
    Practical Assignment 3 <br>
<font color=696880 size=4>
    Amirabbas Afzali

____

# Personal Data

In [ ]:
# Set your student number
student_number = '99105129'
Name = 'Armin'
Last_Name = 'Navardi'

# Rules
- Make sure that all of your cells can be run perfectly.
- Try to minimize your use of ChatGPT (or any other AI assistant) as much as possible.
- You must create a report for this task in PDF format and explain the main results.

---

## **Introduction**

Large Language Models (LLMs) are a class of deep learning models designed for processing and generating natural language. These models are trained using large amounts of textual data and utilize architectures based on transformers. Some of the applications of these models include text generation, machine translation, text summarization, question answering, and text classification.

### *Encoder-Decoder LLMs*

One of the common architectures in large language models is the Encoder-Decoder architecture. In this architecture, the encoder processes an input sequence and maps it to a latent space. Then, the decoder uses this latent space to generate an output sequence. Models like T5 [1] (Text-to-Text Transfer Transformer) use this architecture to perform various tasks. In T5, all tasks are expressed in a "text-to-text" format, meaning both input and output are text. This model has capabilities such as translation, summarization, and text classification. One of the advantages of the Encoder-Decoder architecture is that it allows the encoder to utilize information from both before and after a word to gain a more comprehensive understanding of the text.

### *Decoder-only LLMs*

Decoder-only models, such as GPT-2, GPT-3, and LLaMA [2], unlike the Encoder-Decoder architecture, only use the decoder part. These models use an autoregressive mode, meaning they predict the next token based on previous tokens. These models are highly efficient for text generation and have found widespread applications today.

Advantages of Decoder-only Models

- Efficiency: Decoder-only models are more efficient than Encoder-Decoder models due to the absence of an independent encoder. This makes them require fewer computational and memory resources.
- Simplicity: Due to their autoregressive nature, these models can easily generate sequences in order.
- Scalability: Due to their simpler architecture, these models can be scaled to much larger sizes.


However, one of the drawbacks of these models is that they can only utilize information from tokens before the current token and cannot use tokens that come after for prediction. This limitation is significant in tasks like classification or translation, where a full understanding of the sequence is needed.



## **Objective of the Exercise**


In this exercise, the goal is to convert a generative Decoder-only language model into an encoder and evaluate its performance on a binary sentiment classification task. The main aim is to modify the Decoder-only model so that it can function as an encoder and better handle tasks requiring bidirectional understanding.

## **In this exercise, you should:**


### In this exercise, you should:

1. **Import a Decoder-only model** and load the weights of a pre-trained version of the model.
2. **Generate several outputs from the model**, and include at most 10 sample outputs in your report for different inputs.  
   You should also briefly explain the effects of key configurations in text generation, including:  
   - `Temperature`
   - `top_k`
   - `top_p`
   - `repetition_penalty`
   - `num_beams`
   - `no_repeat_ngram_size`
3. **Load the SST-2 dataset**, which is part of the GLUE benchmark for sentiment classification.  
   - Note that the model’s output depends on the number of input tokens.
   - Apply necessary padding to the dataset after loading it to allow for parallel execution of the model.
4. **Remove the model’s final layer**, which outputs to the size of the model’s dictionary.  
   - Use the embedding vector of the first token (CLS token) for classification.
5. As observed in the previous step, sometimes the embedding vector of the first token does not provide a good representation of the entire input text.  
   - **Add a linear layer** with the same input and output dimensions on top of the encoder's output, and use the output of this linear layer (corresponding to the CLS token) for classification.  
   - This step aggregates information of different tokens to get a comprehensive understanding of the input text.
6. **Instead of the linear layer** in the previous section, use a **bidirectional attention layer** with a custom number of heads (preferably 12).
7. **Repeat step 6** using **left-to-right unidirectional attention** and **right-to-left unidirectional attention**.
8. **Load a pre-trained decoder** (preferably BERT-base) and report its **zero-shot performance** (i.e., without needing to train the model) on the test data.

## **Evaluation:**

In this exercise, for each of sections 4, 5, 6, 7, and 8, you need to plot the confusion matrix corresponding to the model's performance on the test data. Additionally, you should plot two separate graphs showing the training loss and the accuracy of the trained models, and compare them with each other, providing an appropriate analysis of your results. Also, note that high accuracy is not expected for sections 4 and 5, but the correctness of your code will be checked. However, for sections 6 and 7, higher accuracy (around 90%) is expected.













## **Let's go:**


Load `gpt2` model:

In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Model
import torch
from torch import nn

device = "cpu"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Load `sst-2` dataset:

In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

# Load the SST-2 dataset from Hugging Face
dataset = load_dataset("glue", "sst2")

go ahead:

In [3]:
# Generate sample outputs
prompts = ["The movie was", "The product is", "I think it will"]
model.eval()
for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_length=50, temperature=0.7, top_k=50, do_sample=True)
    print(f"Input: {prompt}")
    print(f"Output: {tokenizer.decode(outputs[0], skip_special_tokens=True)}\n")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input: The movie was
Output: The movie was released on DVD in Japan, where it was released on DVD in North America. The movie was a popular hit from the anime and was also a popular manga. The movie was also an internet hit in the United States.

In



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input: The product is
Output: The product is currently under development at Tinkoff Labs, and will be launched on the official website at www.tinkoff.com.

For more information, please check out our FAQs.

Tinkoff Labs is also

Input: I think it will
Output: I think it will help him. He's a good player. I think he's good at what he does. We'll see."

It's the kind of thing that makes me want to believe that this was a huge blowout win.



In [8]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, BertModel, AutoTokenizer
from datasets import load_dataset
from torch.nn import Linear, CrossEntropyLoss
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np


def preprocess_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=MAX_LEN)

BATCH_SIZE = 16
MAX_LEN = 128

tokenizer.pad_token = tokenizer.eos_token
sst2 = dataset.map(preprocess_function, batched=True)
sst2.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

train_loader = DataLoader(sst2["train"], batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(sst2["validation"], batch_size=BATCH_SIZE)


In [ ]:
print()

In [7]:
from transformers import GPT2Model, GPT2Tokenizer, BertForSequenceClassification

# Add a linear layer
class GPT2WithLinearClassifier(torch.nn.Module):
    def __init__(self, gpt2_model):
        super(GPT2WithLinearClassifier, self).__init__()
        self.gpt2 = gpt2_model

        last_layer = nn.Linear(self.gpt2.config.hidden_size, self.gpt2.config.hidden_size, bias=True)

        with torch.no_grad():
          last_layer.weight.copy_(torch.eye(self.gpt2.config.hidden_size))  # Identity matrix
          last_layer.bias.zero_()  # Zero bias

        self.gpt2.lm_head = last_layer

        for param in self.gpt2.parameters():
            param.reguires_grad = False

        self.linear = torch.nn.Linear(self.gpt2.config.hidden_size, 1)
        self.activation = nn.Sigmoid()

        print(self.gpt2.config.hidden_size)

    def forward(self, input_ids, attention_mask=None):
        output = self.gpt2(input_ids, attention_mask=attention_mask)
        cls_embedding = output['logits'][:, 0, :]

        return torch.squeeze(self.activation(self.linear(cls_embedding)))


In [8]:
!pip install torchmetrics
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 11.5 MB/s eta 0:00:00


In [9]:
import tqdm
import torchmetrics

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def train_one_epoch(model, train_loader, loss_fn, optimizer, metric, epoch, device):
  model.train()
  loss_train = AverageMeter()
  metric.reset()

  with tqdm.tqdm(train_loader, unit='batch') as tepoch:
    for batch in tepoch:
      if epoch:
        tepoch.set_description(f'Epoch {epoch}')

      inputs = batch['input_ids'].to(device)
      targets = batch['label'].float().to(device)
      mask = batch['attention_mask'].to(device)

      outputs = model(inputs, mask)

      loss = loss_fn(outputs, targets)

      loss.backward()

      optimizer.step()
      optimizer.zero_grad()

      loss_train.update(loss.item(), n=len(targets))
      metric.update((outputs > 0.5).float(), targets)

      tepoch.set_postfix(loss=loss_train.avg, metric=metric.compute().item())

  return model, loss_train.avg, metric.compute().item()


def evaluate(model, test_loader, loss_fn, metric, device):
    model.eval()
    loss_eval = AverageMeter()
    metric.reset()

    with torch.inference_mode():
      for batch in test_loader:
        inputs = batch['input_ids'].to(device)
        targets = batch['label'].float().to(device)

        outputs = model(inputs)

        loss = loss_fn(outputs, targets)
        loss_eval.update(loss.item(), n=len(targets))

        metric((outputs > 0.5).float(), targets)

    return loss_eval.avg, metric.compute().item()





In [18]:
print(next(iter(train_loader))['label'])

tensor([1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1])


In [10]:
linear_model = GPT2WithLinearClassifier(model).to(device)
loss_fn = nn.BCELoss()
metric = torchmetrics.Accuracy(task="multiclass", num_classes=2).to(device)
optimizer = torch.optim.SGD(linear_model.parameters(), lr=1e-3, momentum=0.9)


loss_train_hist = []
loss_valid_hist = []

metric_train_hist = []
metric_valid_hist = []

best_loss_valid = torch.inf
epoch_counter = 0
num_epochs = 2

for epoch in range(num_epochs):
  linear_model, loss_train, metric_train = train_one_epoch(linear_model, train_loader, loss_fn, optimizer, metric, epoch, device)
  loss_valid, metric_valid = evaluate(linear_model, val_loader, loss_fn, metric, device)

  loss_train_hist.append(loss_train)
  loss_valid_hist.append(loss_valid)

  metric_train_hist.append(metric_train)
  metric_valid_hist.append(metric_valid)

  if loss_valid < best_loss_valid:
    torch.save(linear_model, f'model.pt')
    best_loss_valid = loss_valid
    print('Model Saved!')

  print(f'Valid: Loss = {loss_valid:.4}, Metric = {metric_valid:.4}')
  print()
  epoch_counter += 1

768


  0%|          | 0/4210 [00:04<?, ?batch/s]


KeyboardInterrupt: 

In [17]:
class GPT2_Linear(torch.nn.Module):
    def __init__(self, gpt2_model):
        super(GPT2_Linear, self).__init__()
        self.gpt2 = gpt2_model

        last_layer = nn.Linear(self.gpt2.config.hidden_size, self.gpt2.config.hidden_size, bias=True)

        with torch.no_grad():
          last_layer.weight.copy_(torch.eye(self.gpt2.config.hidden_size))  # Identity matrix
          last_layer.bias.zero_()  # Zero bias

        self.gpt2.lm_head = last_layer

        for param in self.gpt2.parameters():
            param.reguires_grad = False

        self.linear = nn.Sequential(
                    nn.Flatten(),
                    nn.Linear(self.gpt2.config.hidden_size*128, 128),
                    nn.ReLU(),
                    nn.Linear(128, 1),
                    nn.Sigmoid()
        )


        self.activation = nn.Sigmoid()

    def forward(self, input_ids, attention_mask=None):
        output = self.gpt2(input_ids, attention_mask=attention_mask)
        cls_embedding = output['logits']

        return torch.squeeze(self.linear(cls_embedding))

In [ ]:
linear_model = GPT2_Linear(model).to(device)
loss_fn = nn.BCELoss()
metric = torchmetrics.Accuracy(task="multiclass", num_classes=2).to(device)
optimizer = torch.optim.SGD(linear_model.parameters(), lr=1e-3, momentum=0.9)


loss_train_hist = []
loss_valid_hist = []

metric_train_hist = []
metric_valid_hist = []

best_loss_valid = torch.inf
epoch_counter = 0
num_epochs = 2

for epoch in range(num_epochs):
  linear_model, loss_train, metric_train = train_one_epoch(linear_model, train_loader, loss_fn, optimizer, metric, epoch, device)
  loss_valid, metric_valid = evaluate(linear_model, val_loader, loss_fn, metric, device)

  loss_train_hist.append(loss_train)
  loss_valid_hist.append(loss_valid)

  metric_train_hist.append(metric_train)
  metric_valid_hist.append(metric_valid)

  if loss_valid < best_loss_valid:
    torch.save(linear_model, f'model.pt')
    best_loss_valid = loss_valid
    print('Model Saved!')

  print(f'Valid: Loss = {loss_valid:.4}, Metric = {metric_valid:.4}')
  print()
  epoch_counter += 1

  1%|▏         | 61/4210 [21:54<22:55:17, 19.89s/batch, loss=47.1, metric=0.517]

In [ ]:


# Step 5: Use bidirectional attention layer instead of the linear layer
class BidirectionalAttentionClassifier(torch.nn.Module):
    def __init__(self, gpt2_model, num_heads=12):
        super().__init__()
        self.gpt2 = gpt2_model
        self.multihead_attn = torch.nn.MultiheadAttention(self.gpt2.config.hidden_size, num_heads)

    def forward(self, input_ids, attention_mask=None):
        output = self.gpt2(input_ids, attention_mask=attention_mask)
        cls_embedding = get_cls_embedding(output)
        attn_output, _ = self.multihead_attn(cls_embedding.unsqueeze(0),
                                             output.last_hidden_state.transpose(0, 1),
                                             output.last_hidden_state.transpose(0, 1))
        return attn_output.squeeze(0)

bidirectional_model = BidirectionalAttentionClassifier(model)

# Step 6: Repeat using left-to-right unidirectional attention and right-to-left unidirectional attention
class UnidirectionalAttentionClassifier(torch.nn.Module):
    def __init__(self, gpt2_model, direction="left-to-right"):
        super().__init__()
        self.gpt2 = gpt2_model
        self.direction = direction
        self.attn_layer = torch.nn.MultiheadAttention(self.gpt2.config.hidden_size, num_heads=1, add_bias_kv=True)

    def forward(self, input_ids, attention_mask=None):
        output = self.gpt2(input_ids, attention_mask=attention_mask)
        cls_embedding = get_cls_embedding(output)
        if self.direction == "right-to-left":
            reversed_output = torch.flip(output.last_hidden_state, dims=[1])
            attn_output, _ = self.attn_layer(cls_embedding.unsqueeze(0),
                                             reversed_output.transpose(0, 1),
                                             reversed_output.transpose(0, 1))
        else:  # Default left-to-right
            attn_output, _ = self.attn_layer(cls_embedding.unsqueeze(0),
                                             output.last_hidden_state.transpose(0, 1),
                                             output.last_hidden_state.transpose(0, 1))
        return attn_output.squeeze(0)

ltr_model = UnidirectionalAttentionClassifier(model, direction="left-to-right")
rtl_model = UnidirectionalAttentionClassifier(model, direction="right-to-left")

# Step 7: Load pre-trained BERT decoder and report zero-shot performance
bert_model_name = "bert-base-uncased"
bert_model = BertForSequenceClassification.from_pretrained(bert_model_name)

def zero_shot_classification(bert_model, tokenizer, input_text):
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = bert_model(**inputs)
    return torch.nn.functional.softmax(outputs.logits, dim=-1)

# Example test
input_text = "The movie was fantastic and very engaging!"
predictions = zero_shot_classification(bert_model, tokenizer, input_text)
print("Predictions:", predictions)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predictions: tensor([[0.5475, 0.4525]], grad_fn=<SoftmaxBackward0>)


---------
### References

[1] Raffel, Colin, Noam Shazeer, Adam Roberts, et al. (2020). *Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer*. [Link to paper](https://arxiv.org/abs/1910.10683)

[2] Touvron, Hugo, et al. (2023). *LLaMA 2: Open Foundation and Fine-Tuned Chat Models*. [Link to paper](https://arxiv.org/abs/2307.09288)

<span style="color:yellow;">*For further reading on this field of research, you can refer to the following papers:*</span>

[3] BehnamGhader, Adlakha, et al. (2024). *LLM2Vec: Large Language Models Are Secretly Powerful Text Encoders*. [Link to paper](https://arxiv.org/abs/2404.05961)

[4] Gao, Tianyu, et al. (2021). *SimCSE: Simple Contrastive Learning of Sentence Embeddings*. [Link to paper](https://arxiv.org/abs/2104.08821)

[5] Lee, et al. (2023). *NV-Embed: Improved Techniques for Training LLMs as Generalist Embedding Models*. [Link to paper](https://arxiv.org/abs/2405.17428)




# **Best regards.**